In [1]:
import pickle
import numpy
from music21 import instrument, note, stream, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation

Using TensorFlow backend.


In [ ]:
def generate():
    # load notes used for training the model
    with open('notes','rb') as filepath:
        notes = pickle.load(filepath)

    #get all pitch names
    pitchnames = sorted(set(item for item in notes))
    n_vocab = len(set(notes))
    
    network_input, normalized_input = prepare_sequences(notes, pitchnames, n_vocab)
    model = create_network(normalized_input, n_vocab)
    prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)
    create_midi(prediction_output)
    
    

In [ ]:
def prepare_sequences(notes, pitchnames, n_vocab):
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
    
    sequence_length = 100
    network_input = []
    output = []
    for i in range(0,len(notes)-sequence_length, 1):
        sequence_in = notes[i:i+sequence_length]
        sequence_out = notes[i+sequence_length]
        network_input.append([note_to_int[char]] for char in sequence_in)
        output.append(note_to_int[sequence_out])
        
    n_patterns = len(network_output)
    
    #reshape input into a format compatible with LSTM layers
    normalized_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    #normalize input
    normalized_input = normalized_input/(n_vocab)
    
    return(netowork_input)

In [ ]:
#Reuse training code to load weights


def create_network(network_input, n_vocab):
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
    model.add(Dropout(0.3))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(512))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    # Load the weights to each node
    model.load_weights('weights.hdf5')
    
    return model

In [ ]:
# Note Generation using Trained Notes

def note_generation(model, network_input, pitchnames, n_vocab):
    #pick a random note from the NN as a starting point for prediction
    start = numpy.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    #generate 500 notes

    for note_index in range(500):
        prediction_input = numpy.reshape(pattern(1, len(pattern), 1))
        prediction_input = prediction_input/float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = numpy.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)

        pattern.append(index)
        pattern = pattern[1:len(pattern)]
        
    return prediction_output



In [ ]:
# The first sequence we submit is the sequence of notes 
# at the starting index. For every subsequent sequence that 
# we use as input, we will remove the first note of the sequence 
# and insert the output of the previous iteration at the end of 
# the sequence.


# To determine the most likely prediction from the output 
# from the network, we extract the index of the highest value. 
# The value at index X in the output array correspond to the 
# probability that X is the next note.

# First we have to determine whether the output we are decoding is 
# a Note or a Chord.
# If the pattern is a Chord, we have to split the string up 
# into an array of notes. Then we loop through the string 
# representation of each note and create a Note object for 
# each of them. Then we can create a Chord object containing 
# each of these notes.


# If the pattern is a Note, we create a Note object using the 
# string representation of the pitch contained in the pattern.

# At the end of each iteration we increase the offset by 0.5 
# (as we decided in a previous section) and append the Note/Chord 
# object created to a list.

In [ ]:
def create_midi(prediction_output):
    offset = 0
    output_notes = []

    #create note and chord objects - based on values from the model

    for pattern in prediction_output:
        #pattern is a chord
        if('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        #pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        #increase offset each iteration so that the notes do not stack
        offset += 0.5
        midi_stream = stream.Stream(output_notes)
        midi_stream.write('midi', fp='test_output.mid')
        
        
    if __name